In [1]:
import os
import pandas as pd
import numpy as np
import keras

from scipy.sparse.linalg import svds

Using TensorFlow backend.


In [2]:
raw.head()
raw = raw.loc[:, ["buyer", "product", "created"]]
raw["created"] = pd.to_datetime(raw.created)
raw["created"] = [x.month for x in raw["created"]]
raw.columns = ["user", "item", "month"]
raw.shape

(243169, 3)

In [3]:
raw.head()

,user,item,month
0,34990,1047047,7
1,54804,940522,7
2,54804,1029752,7
3,54804,1045597,7
4,54804,1066000,7


In [4]:
raw["month"].unique()

array([ 7,  8,  9, 10, 11, 12], dtype=int64)

In [5]:
i = 0 
s = 7
top_n = 10
user_factor = 10
item_factor = 10

In [6]:
f1_score = []
re_score = []
pr_score = []

In [7]:
while i<=4 :
    raw02 = raw[((raw.month==(s+i))|(raw.month==(s+1+i)))]
    raw02 = raw02.drop_duplicates()
    train = raw02[raw02.month==(s+i)].drop("month",axis=1)
    test = raw02[raw02.month==(s+1+i)].drop("month",axis=1)
    train["rating"] = 1
    test["rat"] = 1
    item_list = train.item.value_counts()[(train.item.value_counts()>=10)].index
    train = pd.pivot_table(train, values="rating", index=["item"], columns=["user"], aggfunc=sum)
    train = train.unstack().reset_index().fillna(0)
    train.columns = ["user","item","rating"]
    train = train[train.item.isin(item_list)]
    train.rating = train.rating.astype("int")
    train["idx"]= "train"
    test = train.merge(test, how = "left", on=["user","item"]).fillna(0)
    test = test[(test.rating!=1)]
    test["rating"] = test.rat.astype("int")
    test = test.drop("rat",axis=1)
    test["idx"] = "test"
    raw02 = pd.concat([train,test])
    raw02_org = pd.concat([train,test])
    raw02["user"] = raw02.user.astype("category").cat.codes.values
    raw02["item"] = raw02.item.astype("category").cat.codes.values
    train = raw02[raw02.idx=="train"]
    train = train.drop("idx",axis=1)
    test = raw02[raw02.idx=="test"]
    test = test.drop("idx",axis=1)
    R_df = train.pivot(index = "user", columns = "item", values = "rating").fillna(0)
    R = R_df.as_matrix()
    user_ratings_mean = np.mean(R, axis = 1)
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
    U, sigma, Vt = svds(R_demeaned, k = 18)
    sigma = np.diag(sigma)
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
    preds_df_stack = preds_df.stack()
    preds_data = pd.DataFrame(preds_df_stack)
    preds_data.reset_index(inplace=True)
    preds_data.columns = ["user", "item", "prob"]
    test["prob"] = preds_data["prob"]
    test = test.reset_index()
    test = test.drop("index",axis=1)
    test01 = test.groupby("user")["prob"].nlargest(top_n).reset_index()
    test01 = test.iloc[list(test01.iloc[ :,1])]
    re = test01.rating.sum() / test.rating.sum()
    pr = test01.rating.sum() / (test.user.nunique() * top_n)
    f1 = 2*(pr*re)  / (pr+re)
    f1_score.append(f1)
    re_score.append(re)
    pr_score.append(pr)
    i+=1

C:\Users\irie9\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [8]:
np.array(f1_score).mean()

0.0026775923227404506

In [9]:
np.array(re_score).mean()

0.2025701151026153

In [10]:
re_score

[0.26666666666666666,
 0.32558139534883723,
 0.16666666666666666,
 0.11469534050179211,
 0.13924050632911392]

In [11]:
np.array(pr_score).mean()

0.0013486710909924654

In [12]:
raw02_org.head()

,user,item,rating,idx
2008,6463,1181679,0,train
2713,6463,1214406,0,train
2947,6463,1223664,0,train
3418,6463,1241443,0,train
3623,6463,1247279,0,train


In [13]:
raw02_org.columns = ["user_org", "item_org", "rating", "idx"]
ref_df = pd.concat([raw02_org, raw02], axis=1)
ref_df = ref_df.drop(["rating", "idx"], axis=1)
ref_df.head()

,user_org,item_org,user,item
2008,6463,1181679,0,0
2713,6463,1214406,0,1
2947,6463,1223664,0,2
3418,6463,1241443,0,3
3623,6463,1247279,0,4


In [14]:
product_dict = dict(zip(ref_df["item"], ref_df["item_org"]))
user_dict = dict(zip(ref_df["user"], ref_df["user_org"]))
recommended_product = test01[["user", "item"]]
recommended_product.reset_index(drop=True, inplace=True)
recommended_product.head()

,user,item
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [15]:
recommended_product["user"] = recommended_product["user"].map(user_dict)
recommended_product["item"] = recommended_product["item"].map(product_dict)

C:\Users\irie9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\irie9\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
# recommended products for July and August
rec_0708 = recommended_product
rec_0708.head()

,user,item
0,6463,1181679
1,6463,1214406
2,6463,1223664
3,6463,1241443
4,6463,1247279
